# ROC Curves in Bokeh

In [1]:
import bokeh.io as bkio
import bokeh.palettes as bkpal
import bokeh.plotting as bkplt
import numpy as np

bkio.output_notebook()

Loading BokehJS ...

In [2]:
np.random.seed(0)
n = 100

true_classes = np.random.randint(2, size=n).astype(np.bool_)
output_probs = np.random.uniform(size=n)
output_probs_2 = np.random.uniform(size=n)

See [Wikipedia's confusion matrix article](https://en.wikipedia.org/wiki/Sensitivity_and_specificity) for the terms I use here.

In [3]:
def bkroc(true_classes,
          output_probs_tuple,
          names=['Classifier 1', 'Classifier 2'],
          cutoffs=np.linspace(0, 1, 10001)):
    n = len(output_probs_tuple)
    colors = bkpal.brewer['Set2'][max(3, n)]
    
    def roc_points(true_classes, output_probs, cutoffs):
        roc_points = []

        for cutoff in cutoffs:
            predicted_condition_positives = (output_probs > cutoff)
            false_positives = (predicted_condition_positives & ~(true_classes))
            condition_negatives = ~true_classes
            fpr = false_positives.sum() / condition_negatives.sum()

            true_positives = (predicted_condition_positives & true_classes)
            condition_positives = true_classes
            tpr = true_positives.sum() / condition_positives.sum()

            roc_points.append([fpr, tpr])

        roc_points = np.array(roc_points)

        return roc_points[:, 0], roc_points[:, 1]
    
    p = bkplt.figure(title='ROC Curve')
    p.xaxis.axis_label = 'False Positive Rate'
    p.yaxis.axis_label = 'True Positive Rate'

    for i in range(n):
        p.line(*roc_points(true_classes, output_probs_tuple[i], cutoffs), line_width=2, legend=names[i], color=colors[i])

    p.legend.location = 'bottom_right'
    p.legend.click_policy = 'hide'

    return p

In [4]:
p = bkroc(true_classes, (output_probs, output_probs_2))
bkplt.show(p)